# **에이전트(Agent) 구축하기**

언어 모델(LLM) 자체는 **행동을 수행할 수 없습니다**  그저 텍스트를 출력할 뿐입니다.  
LangChain의 주요 사용 사례 중 하나는 **에이전트(Agents)**를 만드는 것입니다.  

**에이전트(Agents)**는 **LLM**을 **추론 엔진**으로 사용하여 어떤 행동을 취해야 하는지, 그리고 행동을 수행하는 데 필요한 입력이 무엇인지 결정하는 시스템입니다.  
행동을 실행한 후 결과를 LLM에 다시 전달하여 **더 많은 행동이 필요한지** 또는 **작업을 종료해도 되는지**를 판단할 수 있습니다. 이는 종종 **도구 호출(tool-calling)**을 통해 이루어집니다.  

---

이 노트북에서는 **검색 엔진과 상호작용할 수 있는 에이전트**를 구축합니다.  
- 사용자는 이 에이전트에게 질문을 할 수 있습니다.  
- 에이전트가 검색 도구를 호출하는 과정을 볼 수 있습니다.  
- 에이전트와 여러 차례 대화를 나눌 수 있습니다.  

---

## **엔드 투 엔드 에이전트 (End-to-end Agent)**

아래 코드는 **도구(tool)**를 사용하여 어떤 행동을 취해야 할지 결정하는 LLM을 활용한 **완전히 기능하는 에이전트**를 보여줍니다.  
- 이 에이전트는 **일반적인 검색 도구**를 사용할 수 있습니다.  
- **대화 메모리(conversational memory)**를 갖추고 있어 다중 턴 대화가 가능합니다.
- 도구 정의 - 우리의 주요 선택 도구는 **Tavily**입니다. Tavily는 **검색 엔진**으로, LangChain에는 Tavily 검색 엔진을 쉽게 사용할 수 있도록 지원하는 **내장 도구**가 포함되어 있습니다.


In [1]:
# !pip install -qU \
# python-dotenv \
# langchain \
# langchain-community \
# openai \
# anthropic \
# langchain-openai \
# langchain-anthropic \
# langchain-google-genai \
# python-dotenv \
# langgraph

In [2]:
# !pip install -qU tavily-python langgraph-checkpoint-sqlite

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
# LangChain 추적(Tracing) 설정 활성화
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [5]:
from langchain_openai import ChatOpenAI
# from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

##  **언어 모델과 도구 정의**

먼저 언어 모델과 사용하려는 도구를 생성해야 합니다. 우리의 주요 선택 도구는 **Tavily**입니다. Tavily는 **검색 엔진**으로, LangChain에는 Tavily 검색 엔진을 쉽게 사용할 수 있도록 지원하는 **내장 도구**가 포함되어 있습니다.

In [6]:
# model = ChatAnthropic(model="claude-3-5-sonnet-20241022")
model = ChatOpenAI(model="gpt-4o-mini")

# 검색 도구 설정 (최대 검색 결과 2개)
search = TavilySearchResults(max_results=2)

# 사용할 모든 도구를 하나의 리스트에 넣어서 나중에 참조할 수 있도록 설정
tools = [search]
tools

[TavilySearchResults(max_results=2, api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********')))]

### 1) 도구 없이 LLM 만 호출할 경우

In [7]:
print(model.invoke("안녕, 나는 서울에 살고 있어. 내가 사는 곳의 날씨를 알려줘").content)

안녕하세요! 서울의 현재 날씨를 실시간으로 제공할 수는 없지만, 일반적으로 서울의 날씨는 계절에 따라 크게 다릅니다. 봄과 가을에는 온화한 날씨가 많고, 여름에는 덥고 습한 날이 많으며, 겨울에는 춥고 눈이 오는 날도 많습니다. 정확한 날씨 정보를 원하시면 기상청 웹사이트나 날씨 앱을 확인해 보시는 것을 추천합니다. 도움이 필요하시면 말씀해 주세요!


### 2) Agent 구축 않고 LLM에 도구 목록만 전달할 경우
이제 이 모델이 도구 호출을 수행하도록 활성화하기 위해 `.bind_tools`를 사용하여 언어 모델에 이러한 도구에 대한 지식을 제공합니다.

In [8]:
# `bind_tools` 메서드를 사용하여 언어 모델이 특정 도구 목록을 사용할 수 있도록 설정합니다.
model_with_tools = model.bind_tools(tools)

response = model_with_tools.invoke([HumanMessage(content="안녕, 나는 서울에 살고 있어. 내가 사는 곳의 날씨를 알려줘")])
print(response)

content='' additional_kwargs={'tool_calls': [{'id': 'call_Ky8RWC3mEGTWMEIbHUAJLWMJ', 'function': {'arguments': '{"query":"서울 날씨"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 99, 'total_tokens': 120, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_d02d531b47', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-b1661db9-fbf3-4b1a-9dda-d64b81ce4745-0' tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '서울 날씨'}, 'id': 'call_Ky8RWC3mEGTWMEIbHUAJLWMJ', 'type': 'tool_call'}] usage_metadata={'input_tokens': 99, 'output_tokens': 21, 'total_tokens': 120, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 

In [9]:
print(response.tool_calls)

[{'name': 'tavily_search_results_json', 'args': {'query': '서울 날씨'}, 'id': 'call_Ky8RWC3mEGTWMEIbHUAJLWMJ', 'type': 'tool_call'}]


모델은 우리가 **Tavily Search** 도구를 호출하라고 알려주며 도구 호출에 필요한 argument를 만들어 줍니다.  
하지만 이것은 아직 해당 도구를 호출한 것이 아니라, 단지 호출해야 한다고 알려주는 것입니다. 실제로 도구를 호출하려면 **에이전트(agent)**를 만들어야 합니다.

## Agent 생성

- LLM은 **상태를 저장하지 않는(stateless)** 이므로 이전 상호작용을 기억하지 않습니다. 에이전트에 메모리를 추가하려면 **체크포인터(checkpointer)**를 전달해야 합니다. 또한 에이전트를 호출할 때 `thread_id`를 함께 전달해야 합니다. 이를 통해 에이전트는 어느 스레드(대화)에서부터 상호작용을 이어가야 하는지 알 수 있습니다.

- ReAct 에이전트를 구성하기 위해 Langgraph에서 제공하는 고수준 인터페이스(high-level interface)인 `create_react_agent`를 사용합니다. 이를 통해 에이전트 로직을 원하는 대로 세부적으로 수정할 수 있습니다. `create_react_agent`는 내부적으로 `.bind_tools`를 호출해 도구를 모델에 연결합니다.

In [10]:
# 에이전트의 메모리 저장 객체 생성
memory = MemorySaver()  

# 에이전트 실행 객체 생성
agent_executor = create_react_agent(model, tools, checkpointer=memory)

# 에이전트 실행 구성 설정
config = {"configurable": {"thread_id": "abc123"}}

## 스트리밍 메시지  

우리는 `.invoke`를 사용해 에이전트를 호출하고 최종 응답을 받는 방법을 알고 있습니다. 하지만 에이전트가 여러 단계를 실행해야 할 경우 시간이 오래 걸릴 수 있습니다.  
중간 진행 상황을 보여주기 위해, 발생하는 메시지를 스트리밍 형태로 실시간 전송할 수 있습니다.

In [11]:
# 에이전트 실행
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="안녕! 나는 길동이야. 서울에 살고 있어.")]},
    config
):
    print(chunk)  # 에이전트의 응답 출력
    print("---------------------------------------------------------------------")  # 응답 구분선 출력

{'agent': {'messages': [AIMessage(content='안녕, 길동! 서울에 살고 있다니 멋지네. 서울에서 어떤 것들을 좋아해?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 94, 'total_tokens': 121, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None}, id='run-7541a69e-5ce4-461b-b317-5604230436aa-0', usage_metadata={'input_tokens': 94, 'output_tokens': 27, 'total_tokens': 121, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
---------------------------------------------------------------------


#### 메모리가 필요한 질문 

In [12]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="내 이름이 뭐고 어디에 살고 있다고 했지?")]},
    config
):
    print(chunk)  # 에이전트의 응답 출력
    print("------------------------------------------------------------------------")  # 응답 구분선 출력

{'agent': {'messages': [AIMessage(content='너의 이름은 길동이고, 서울에 살고 있다고 했어. 맞아?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 140, 'total_tokens': 161, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None}, id='run-eba1dca8-290d-46d4-b48a-998fb466bd67-0', usage_metadata={'input_tokens': 140, 'output_tokens': 21, 'total_tokens': 161, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
------------------------------------------------------------------------


#### ReAct 패턴의 질문/응답 
질문이 단순 정보 조회가 아닌, 추론과 행동을 반복해야 해결 가능한 질문
예) 내가 사는 곳의 날씨를 알려줘 
- Reasoning(추론) - 사용자의 위치를 알아야 하고, 날씨 API에서 날씨 정보를 가져와야 한다. (LLM 역할)  
- Acting(행동) - TavilySearchResults API 호출 (LangChin 역할)  
- Observation(관찰) - API로부터 "오늘 서울은 맑음"이라는 정보를 받았다. 이 것을 LLM에 다시 전달. (LangChain 역할)
- Reasoning(추론) - "이제 사용자에게 서울의 날씨가 맑다고 답변해야 한다."  (LLM의 역할)  
- Answer(응답) - "서울의 날씨는 맑음입니다." (LLM의 역할)  

In [13]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="내가 사는 곳의 날씨를 알려줘.")]},
    config
):
    print(chunk)  # 에이전트의 응답 출력
    print("-----------------------------------------------------------------------")  # 응답 구분선 출력

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ddBYt55HfLgXwfnMH3PPbm4s', 'function': {'arguments': '{"query":"서울 날씨"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 179, 'total_tokens': 200, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-adf4279e-866c-4c60-a14d-f29b293b3f04-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '서울 날씨'}, 'id': 'call_ddBYt55HfLgXwfnMH3PPbm4s', 'type': 'tool_call'}], usage_metadata={'input_tokens': 179, 'output_tokens': 21, 'total_tokens': 200, 'input_token_details': {'audio': 0, 'cache_read

Example [LangSmith trace](https://smith.langchain.com/o/351c6cd9-1396-5c74-9478-1ee6a22a6433/projects/p/acec9d4d-4978-4597-adff-789cd42e200f?timeModel=%7B%22duration%22%3A%227d%22%7D&peek=17495aca-857d-4d57-95e7-6c05ac8348d7)

새로운 대화를 시작하려면 사용 중인 `thread_id`를 변경하기만 하면 됩니다.

In [14]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="내 이름이 뭐야?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='죄송하지만, 당신의 이름을 알 수 있는 방법이 없습니다. 이름을 알려주시면 그에 대한 질문이나 대화를 이어갈 수 있습니다!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 85, 'total_tokens': 119, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None}, id='run-edd2dce7-9a71-41d0-9534-3880d05ec8a1-0', usage_metadata={'input_tokens': 85, 'output_tokens': 34, 'total_tokens': 119, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----
